In [1]:
from dolfinx.mesh import create_rectangle
from mpi4py import MPI

from dolfinx.fem import function, functionspace
from basix.ufl import element, mixed_element


In [2]:
from MultiphysicsModel.Mesh import RBMesh
import numpy as np

a = 1.44e-3

# Lower left and upper right edges
points = ((-2*a,-2*a),(2*a,2*a))

# Number of elements
n = (100,100)

markers = {
    "leftright":    lambda x: np.isclose(x[0], -2*a) | np.isclose(x[0],2*a),
    "top":          lambda x: np.isclose(x[1], 2*a),
    "bottom":       lambda x: np.isclose(x[1], -2*a),
}

mesh = RBMesh(points=points, n=n, bc_markers=markers)

In [16]:
from MultiphysicsModel.FEData import FEData

fe_config = {

    "alpha1": {
        "element": "DG", 
        "degree": 1, 
        "type": "scalar",
        "time_scheme":  "explicit euler",
    },

    "alpha2": {
        "element": "DG", 
        "degree": 1, 
        "type": "scalar",
        "time_scheme":  "explicit euler",
        },

    "p": {
        "element": "CG", 
        "degree": 1, 
        "type": "scalar",
        "time_scheme":  "explicit euler",
        },

    "u": {
        "element": "CG", 
        "degree": 2, 
        "type": "vector",
        "time_scheme":  "explicit euler",
        },

    "T": {
        "element": "CG", 
        "degree": 1, 
        "type": "scalar",
        "time_scheme":  "explicit euler",
        },
}

fe_data = FEData(mesh=mesh, config=fe_config, create_mixed=True)

KeyError: 'alpha_solid'

In [8]:
cell = mesh.dolfinx_mesh.basix_cell()
fe_T        = element("P", cell, degree=1, discontinuous=False)
fe_p        = element("P", cell, degree=1, discontinuous=True)
fe_u        = element("P", cell, degree=1, discontinuous=False, shape=(mesh.cell_dim,))
fe_alpha    = element("P", cell, degree=1, discontinuous=True)

fe_mixed = mixed_element([fe_alpha,fe_u,fe_p,fe_T])

In [9]:
fs = functionspace(mesh=mesh.dolfinx_mesh,element=fe_mixed)
fs_idxs = {     # Maps the field name to the sub-index of the compound function space
    "alpha":    0,
    "u":        1,
    "p":        2,
    "T":        3,
}

According to [this discussion](https://fenicsproject.discourse.group/t/cannot-find-dofs-for-discontinuous-element/10900/3), discontinuous spaces do not have DoFs that live on a facet. Thus, `locate_dofs_topological` does not find any DoFs there. Must rely on `locate_dofs_geometrical` for now.

In [10]:
from dolfinx.fem import locate_dofs_geometrical

boundary_dofs = {}

for (field,idx) in fs_idxs.items():
    boundary_dofs[field] = {}
    sub = fs.sub(idx)
    space, _ = sub.collapse()
    for (facet,marker) in markers.items():
        boundary_dofs[field][facet] = locate_dofs_geometrical(
            V=(sub,space),
            marker=marker
        )

boundary_dofs["u_x"] = {}
sub = fs.sub(fs_idxs["u"]).sub(0)
space, _ = sub.collapse()
for (facet,marker) in markers.items():
        boundary_dofs["u_x"][facet] = locate_dofs_geometrical(
            V=(sub,space),
            marker=marker
        )


In [11]:
from dolfinx.fem import Function

T_l = 290.000 # K
T_u = 291.152 # K

T_bottom = Function(fs.sub(fs_idxs["T"]).collapse()[0])
T_top = Function(fs.sub(fs_idxs["T"]).collapse()[0])

T_bottom.interpolate(lambda x: np.full(x.shape[0],T_l))
T_top.interpolate(lambda x: np.full(x.shape[0],T_u))

u_x_bc = Function(fs.sub(fs_idxs["u"]).sub(0).collapse()[0])    # x component of velocity
u_x_bc.interpolate(lambda x: np.full(x.shape[0], (0.)))

p_bc = Function(fs.sub(fs_idxs["p"]).collapse()[0])
p_bc.interpolate(lambda x: np.full(x.shape[0],0.0))

In [12]:
from dolfinx.fem import dirichletbc

bcs = {}

bcs["T"] = [
    dirichletbc(value=T_bottom, dofs=boundary_dofs["T"]["bottom"],V=fs.sub(fs_idxs["T"])),
    dirichletbc(value=T_top, dofs=boundary_dofs["T"]["top"],V=fs.sub(fs_idxs["T"])),
]

bcs["u"] = [
    # Symmetry BC: no penetration through wall, tangential slip
    dirichletbc(value=u_x_bc, dofs=boundary_dofs["u_x"]["leftright"],V=fs.sub(fs_idxs["u"]).sub(0)),
    # No slip
    dirichletbc(value=u_x_bc, dofs=boundary_dofs["u_x"]["top"],V=fs.sub(fs_idxs["u"]).sub(0)),
    # No slip
    dirichletbc(value=u_x_bc, dofs=boundary_dofs["u_x"]["bottom"],V=fs.sub(fs_idxs["u"]).sub(0)),
]

bcs["p"] = [
    dirichletbc(value=p_bc, dofs=boundary_dofs["p"]["leftright"], V=fs.sub(fs_idxs["p"]))
]

In [13]:
from MultiphysicsModel.TimeDependentFunction import TimeDependentFunction

solution = TimeDependentFunction(previous=Function(fs),
                                 current=Function(fs))

functions = {}
for (field, idx) in fs_idxs.items():
    functions[field] = TimeDependentFunction(
        previous=solution.previous.sub(idx),
        current=solution.current.sub(idx)
        )

In [10]:
def IC_alpha1(x):
            return np.where(np.sqrt(x[0]**2+x[1]**2) <= a, 1.0, 0.0)
functions["alpha"].previous.interpolate(IC_alpha1)

def IC_T(x):
        out = np.zeros(x.shape[0])
        out[:] = (T_l+T_u)/2.0 + (T_u-T_l)/(4.0*a) * x[:,1]
        return out
functions["T"].previous.interpolate(IC_T)

In [11]:
from MultiphysicsModel.FEOperators import FEOperators

ops = FEOperators(mesh=mesh)

AttributeError: 'Mesh' object has no attribute 'dolfinx_mesh'